In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
ticker = yf.Ticker("RELI")
data=ticker.history(start="2022-01-01",end="2024-12-31",interval="1d")
data = data.dropna()
print(data.head())


                                  Open         High          Low        Close  \
Date                                                                            
2022-01-03 00:00:00-05:00  1660.050049  2435.250000  1606.500000  1932.900024   
2022-01-04 00:00:00-05:00  1782.449951  2572.949951  1736.550049  2376.600098   
2022-01-05 00:00:00-05:00  2152.199951  2674.949951  2073.149902  2269.500000   
2022-01-06 00:00:00-05:00  2286.840088  2437.800049  1657.500000  1721.250000   
2022-01-07 00:00:00-05:00  1731.449951  1884.449951  1448.400024  1555.500000   

                           Volume  Dividends  Stock Splits  
Date                                                        
2022-01-03 00:00:00-05:00  100015        0.0           0.0  
2022-01-04 00:00:00-05:00  204411        0.0           0.0  
2022-01-05 00:00:00-05:00   94822        0.0           0.0  
2022-01-06 00:00:00-05:00   25985        0.0           0.0  
2022-01-07 00:00:00-05:00   15810        0.0           0.0  


In [ ]:
#bb
def bb(data,window=20):
  data["sma"]= data["Close"].rolling(window).mean()
  data["std"]= data["Close"].rolling(window).std()
  data["upperband"]=data["sma"]+2*(data["std"])
  data["lowerband"]=data["sma"]-2*(data["std"])
  return data
#macd
def macd(data,window_signal=9):
  data["ema_slow"]=data["Close"].ewm(span=26,adjust=False).mean()
  data["ema_fast"]=data["Close"].ewm(span=12,adjust=False).mean()
  data["signal"]=data["Close"].ewm(span=window_signal,adjust=False).mean()
  data["macd"]=data["ema_fast"]-data["ema_slow"]
  return data

def rsi(data,period=14):
  delta = data['Close'].diff()
  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)
  avg_gain = gain.rolling(window=period).mean()
  avg_loss = loss.rolling(window=period).mean()
  rs = avg_gain / avg_loss
  data['RSI'] = 100 - (100 / (1 + rs))
  return data
#atr
def atr(data,period=14):
  data["h-l"] = data["High"]-data["Low"]
  data["h-pc"] = abs(data["High"]-data["Close"].shift(1))
  data["l-pc"] = abs(data["Low"]-data["Close"].shift(1))
  data["tr"]=data[["h-l","h-pc","l-pc"]].max(axis=1)
  data["atr"]=data["tr"].rolling(period).mean()
  return data
def sc(data):
  period = 14
  low_min = data['Low'].rolling(window=period).min()
  high_max = data['High'].rolling(window=period).max()
  data['%K'] = 100 * (data['Close'] - low_min) / (high_max - low_min)
  data['%D'] = data['%K'].rolling(window=3).mean()
  return data
def vwap(data):
  data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3
  data['TPV'] = data['Typical_Price'] * data['Volume']
  data['Cum_TPV'] = data['TPV'].cumsum()
  data['Cum_Volume'] = data['Volume'].cumsum()
  data['VWAP'] = data['Cum_TPV'] / data['Cum_Volume']
  return data

vwap(data)
bb(data)
atr(data)
rsi(data)
macd(data)
sc(data)



,Open,High,Low,Close,Volume,Dividends,Stock Splits,TPV,VWAP,sma,...,ema_fast,signal,macd,%K,%D,Typical_Price,Cum_TPV,Cum_Volume,low_min,high_max
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03 00:00:00-05:00,1660.050049,2435.250000,1606.500000,1932.900024,100015,0.0,0.0,1.991849e+08,1991.550008,NaN,...,1932.900024,1932.900024,0.000000,NaN,NaN,1991.550008,1.991849e+08,100015,NaN,NaN
2022-01-04 00:00:00-05:00,1782.449951,2572.949951,1736.550049,2376.600098,204411,0.0,0.0,4.555708e+08,2150.787634,NaN,...,2001.161574,2021.640039,35.394878,NaN,NaN,2228.700033,6.547557e+08,304426,NaN,NaN
2022-01-05 00:00:00-05:00,2152.199951,2674.949951,2073.149902,2269.500000,94822,0.0,0.0,2.218076e+08,2195.535843,NaN,...,2042.444409,2071.212031,54.178949,NaN,NaN,2339.199951,8.765633e+08,399248,NaN,NaN
2022-01-06 00:00:00-05:00,2286.840088,2437.800049,1657.500000,1721.250000,25985,0.0,0.0,5.038102e+07,2179.850369,NaN,...,1993.029884,2001.219625,24.543348,NaN,NaN,1938.850016,9.269443e+08,425233,NaN,NaN
2022-01-07 00:00:00-05:00,1731.449951,1884.449951,1448.400024,1555.500000,15810,0.0,0.0,2.576160e+07,2160.120252,NaN,...,1925.717595,1912.075700,-12.177347,NaN,NaN,1629.449992,9.527059e+08,441043,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23 00:00:00-05:00,3.840000,5.110000,2.810000,3.110000,105557300,0.0,0.0,3.880990e+08,16.315084,1.55765,...,1.645824,1.695819,0.002016,49.174075,19.684470,3.676667,2.740983e+09,168002988,1.175,5.11
2024-12-24 00:00:00-05:00,3.000000,3.130000,2.450000,2.530000,1590800,0.0,0.0,4.300463e+06,16.187405,1.60415,...,1.781851,1.862655,0.072399,34.434561,29.344477,2.703333,2.745283e+09,169593788,1.175,5.11
2024-12-26 00:00:00-05:00,2.430000,2.920000,2.430000,2.660000,739900,0.0,0.0,1.975533e+06,16.128687,1.66565,...,1.916951,2.022124,0.137088,37.738248,40.448961,2.670000,2.747259e+09,170333688,1.175,5.11
